In [1]:
import os,sys
sys.path.append(os.getcwd())

#Module imports
import geeViz.getImagesLib as getImagesLib
ee = getImagesLib.ee
Map = getImagesLib.Map

print('done')

/Users/jessieeastburn/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Initializing GEE
Successfully initialized
geeViz package folder: /Users/jessieeastburn/Library/Python/3.9/lib/python/site-packages/geeViz
done


In [2]:
import pandas as pd
import time
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

/var/folders/sf/r2b9cx3s6fl_d5hscrs0_44c0000gn/T/ipykernel_97938/518313686.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
# Get utka polygon.
studyArea = (
    ee.FeatureCollection('projects/disco-glass-413820/assets/plot_min_bound_geom')
    .filter("site_id == 'UOFU_UTKA'")
    .first()
    .geometry()
)

Map.port = 1992
Map.clearMap()

studyArea = ee.Geometry.Polygon(
            [[[-112.2647, 37.02876], 
              [-112.2647, 37.10894],
              [-112.0867, 37.10894],
              [-112.0867, 37.02876], 
              [-112.2647, 37.02876]]])
Map.centerObject(studyArea, 11)
Map.addLayer(studyArea, {'color': 'blue'}, 'utka')
Map.view()


Adding layer: utka
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:1992/geeView/ already serving.
cwd /Users/jessieeastburn/Library/CloudStorage/OneDrive-UniversityofUtah/URSA_Work/TS2/PJ_TimeSeries
geeView URL: http://localhost:1992/geeView/?projectID=None&accessToken=ya29.a0AfB_byD8vaCIJorViR0m53PrxDtT2A5uHHYDCDy5nin9iugFs5dY5mKxTrQLTjjY8_qH7lwL6IEJ7_ffM4SzXiZoGgGFBYr63FKMFQBFwavPXCC6Y2SxIExittWoVFfZzgx2g_EkBjihPHJRVOXJ8aQ03MrbIeb0pkFOegaCgYKAWUSARISFQHGX2MiV09TnLjR_C8aSheIbz-Q1Q0173


::1 - - [21/Feb/2024 15:03:20] "GET /geeView/?projectID=None&accessToken=ya29.a0AfB_byD8vaCIJorViR0m53PrxDtT2A5uHHYDCDy5nin9iugFs5dY5mKxTrQLTjjY8_qH7lwL6IEJ7_ffM4SzXiZoGgGFBYr63FKMFQBFwavPXCC6Y2SxIExittWoVFfZzgx2g_EkBjihPHJRVOXJ8aQ03MrbIeb0pkFOegaCgYKAWUSARISFQHGX2MiV09TnLjR_C8aSheIbz-Q1Q0173 HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:03:20] "GET /geeView/src/gee/gee-run/runGeeViz.js HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:03:20] "GET /geeView/src/js/load.min.js HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:03:20] "GET /geeView/src/styles/style.min.css HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:03:20] "GET /geeView/src/js/gena-gee-palettes.js HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:03:20] "GET /geeView/src/js/lcms-viewer.min.js HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:03:20] "GET /geeView/src/assets/images/GEE_logo_transparent.png HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:03:20] "GET /geeView/src/assets/images/usfslogo.png HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:03:20] "GET /geeView/src/assets/images

In [7]:
# Run get images, set parameters manually, and export
lsAndTs = getImagesLib.getLandsatWrapper(\

    # Study area and time
    studyArea = studyArea,
    startYear = 2015,
    endYear = 2020,
    startJulian = 1,
    endJulian = 365,

    # Moving window options
    timebuffer =  0,# If multi-year moving window is needed, use timebuffer > 0 (e.g. timebuffer = 1 will result in a 3 year moving window)
    weights =  [1],# To compliment the timebuffer, set weights for each year of the moving window. This enables the center year to have more weight (e.g. [1,3,1] would weight the center year if timebuffer = 1 3x over the buffer years)

    # Composite methods and input imagery
    compositingMethod = 'medoid', # Specify median or medoid. Medoid allows for storing only the spectral and ancillary bands without indices
    toaOrSR = 'SR', # It is advised to only use TOA when combining Landsat and Sentinel-2 since the Sentinel-2 SR data in GEE are terrain-corrected and Landsat are not
    includeSLCOffL7 = False, # Whether to include Landsat 7 after May 2003. Scanline artifacts can be found in composites when True
    defringeL5 = True, # Whether to get rid of fringe edges found mostly on Landsat 4 and 5
    landsatCollectionVersion = 'C2', # To use Collection 1 or 2. Only use C1 if doing legacy work prior to 2022

    # Cloud and shadow masking
    # There will be a parameter for applying CloudScore+ to Sentinel-2 provided once it is available for all S2 data
    applyCloudScore = True, # We use this since fMask struggles for cloud masking
    applyTDOM = True, # We use TDOM for Landsat since the fMask cloud shadow masking method struggles a lot
    applyFmaskCloudMask = True,
    applyFmaskCloudShadowMask = True,
    applyFmaskSnowMask = False,
    cloudScoreThresh = 10,# The threshold applied to the cloudScore for Landsat and/or Sentinel-2
    performCloudScoreOffset = False,# Set this to False in areas with limited data and/or limited bright cool surfaces
    cloudScorePctl = 10, # If performCloudScoreOffset is set to True, this is the percentile of the cloudScores used to see if an area typically has a high cloudScore
    zScoreThresh = -1, # The TDOM Z-score threshold for finding dark outliers
    shadowSumThresh = 0.35,# The sum of the shadowSumBands has to be below this to be considered dark
    contractPixels = 1.5,# Inward buffer pixel radius to get rid of salt and pepper
    dilatePixels = 3.5, # Outwoard buffer pixel radius to dilate the cloud shadow mask by

    # Image processing
    correctIllumination = False, # Method for illuminating hill shadows. This is largely deprecated and not used
    correctScale = 250,

    # Export parameters
   # exportComposites = False,# Whether to export resulting composites exportCompositeCollection
   # exportPathRoot = 'projects/disco-glass-413820/assets/users/lcms-training_module-2_composites', #'content/drive/MyDrive/lcms-training_module-2_composites',used this i think #'projects/resolute-future-412019/assets/users/lcms-training_module-2_composites', og directory #'projects/earthengine-public/assets/content/drive/MyDrive/lcms-training_module-2_composites',
    exportComposites = False,
    #outputName = 'Landsat-UTKA',
    
    crs = 'EPSG:4326',
    #transform = getImagesLib.common_projections['NLCD_CONUS']['transform'],
    transform = None)
 #  scale = None,
  #  overwrite = True,# Whether to overwrite existing composites

    #Separate into scenes and composites for subsequent analysis
processedScenes = lsAndTs['processedScenes']
processedComposites = lsAndTs['processedComposites']

#clear map
Map.clearMap()
#Turn off layers from previous iteration
Map.turnOffAllLayers()
startYear = 2015
endYear = 2020
#Map.addLayer(processedComposites.select(['NDVI','NBR']),{'addToLegend':'false'},'Time Series (NBR and NDVI)',True)
for year in range(startYear,endYear + 1 ):
     t = processedComposites.filter(ee.Filter.calendarRange(year,year,'year')).mosaic()
     Map.addLayer(t.float(),getImagesLib.vizParamsFalse,'UTKA{}'.format(year),'True')

Map.turnOnInspector()
Map.view()
print('Done')

Get Processed Landsat: 
Start date: Jan 01 2015 , End date: Dec 30 2020
Applying scale factors for C2 L4 data
Applying scale factors for C2 L5 data
Defringing L4 and L5
Applying scale factors for C2 L8 data
Only including SLC On Landsat 7
Applying scale factors for C2 L7 data
Applying scale factors for C2 L9 data
Applying Cloud Score
Not computing cloudScore offset
Applying Fmask Cloud Mask
Applying TDOM Shadow Mask
Computing irMean for TDOM
Computing irStdDev for TDOM
Applying Fmask Shadow Mask
Adding layer: UTKA2015
Adding layer: UTKA2016
Adding layer: UTKA2017
Adding layer: UTKA2018
Adding layer: UTKA2019
Adding layer: UTKA2020
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:1992/geeView/ already serving.
cwd /Users/jessieeastburn/Library/CloudStorage/OneDrive-UniversityofUtah/URSA_Work/TS2/PJ_TimeSeries
geeView URL: http://localhost:1992/geeView/?projectID=None&accessToken=ya29.a0AfB_byAyWp6rkD4xmBLlR8urwaVRe0ouYFHcmwEAFSXKAVsebz8qeS8vI

Done


::1 - - [21/Feb/2024 15:03:47] "GET /geeView/?projectID=None&accessToken=ya29.a0AfB_byAyWp6rkD4xmBLlR8urwaVRe0ouYFHcmwEAFSXKAVsebz8qeS8vI_-yYgGO4tz9jRNX0FJNmuUA8USOk_xP4qMyLfgf9Lg9PSLbyY7v5qv21I1Vfnh6APCCekadOwe4FsU2SmEOIy7mE0Gr4ctTl05HqU9I4JZDmwaCgYKAb8SARISFQHGX2MijOpkT6F953sRBkzMmGTWRQ0173 HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:03:47] "GET /geeView/src/gee/gee-run/runGeeViz.js HTTP/1.1" 200 -


In [8]:
#Clip composites to study area
clipped_collection = processedComposites.map(lambda image: image.clip(studyArea))

In [10]:
#clear map
Map.clearMap()
#Turn off layers from previous iteration
Map.turnOffAllLayers()
#Map.addLayer(processedComposites.select(['NDVI','NBR']),{'addToLegend':'false'},'Time Series (NBR and NDVI)',True)
for year in range(startYear,endYear + 1 ):
     t = clipped_collection.filter(ee.Filter.calendarRange(year,year,'year')).mosaic()
     Map.addLayer(t.float(),getImagesLib.vizParamsFalse,'UTKA{}'.format(year),'True')
Map.turnOnInspector()
Map.view()
print('Done')

Adding layer: UTKA2015
Adding layer: UTKA2016
Adding layer: UTKA2017
Adding layer: UTKA2018
Adding layer: UTKA2019
Adding layer: UTKA2020
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:1992/geeView/ already serving.
cwd /Users/jessieeastburn/Library/CloudStorage/OneDrive-UniversityofUtah/URSA_Work/TS2/PJ_TimeSeries
geeView URL: http://localhost:1992/geeView/?projectID=None&accessToken=ya29.a0AfB_byB4ZTaHAQ7X48F-9iUoPN3qAZ164q9iDkLB9SyaJaKS362sSV-mc7LW-xM98jdM6SizToX_XD9Y94s6OUDUnLH00ASW-gvlHGztuIu7Lv8e2Q4eQEQfjML1n79ovVV1Zy20kMF2n54ulH-yETHkRXR85LM9J-XXQgaCgYKARoSARISFQHGX2Mi_zp_qLxOLdMmBze9tSYL2g0173


Done


::1 - - [21/Feb/2024 15:07:24] "GET /geeView/?projectID=None&accessToken=ya29.a0AfB_byB4ZTaHAQ7X48F-9iUoPN3qAZ164q9iDkLB9SyaJaKS362sSV-mc7LW-xM98jdM6SizToX_XD9Y94s6OUDUnLH00ASW-gvlHGztuIu7Lv8e2Q4eQEQfjML1n79ovVV1Zy20kMF2n54ulH-yETHkRXR85LM9J-XXQgaCgYKARoSARISFQHGX2Mi_zp_qLxOLdMmBze9tSYL2g0173 HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:07:24] "GET /geeView/src/gee/gee-run/runGeeViz.js HTTP/1.1" 200 -


In [11]:
#LandTrendr
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
import geeViz.gee2Pandas as g2p

import inspect,os
#resolute-future-412019
ee = getImagesLib.ee
Map = getImagesLib.Map

print('Done')

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.
Done


In [12]:
lcms_actual_lt_collection = ee.ImageCollection('projects/lcms-tcc-shared/assets/OCONUS/R8/PR_USVI/Base-Learners/LandTrendr-Collection')
print('All bands LCMS uses for LandTrendr:',lcms_actual_lt_collection.aggregate_histogram('band').keys().getInfo())

All bands LCMS uses for LandTrendr: ['NBR', 'NDMI', 'NDSI', 'NDVI', 'blue', 'brightness', 'green', 'greenness', 'nir', 'red', 'swir1', 'swir2', 'tcAngleBG', 'wetness']


In [13]:
# Bring in composites and pull info from them
composites = ee.ImageCollection(clipped_collection) #mine
props = composites.first().toDictionary().getInfo()

startYear = props['startYear']
endYear = props['endYear']

startJulian = props['startJulian']
endJulian = props['endJulian']

proj = composites.first().projection().getInfo()

# Pull out the crs
# Depending on if a wkt or epsg format is used, it will be stored under a different key
if 'crs' not in proj.keys():
    crs = proj['wkt']
else:
    crs = proj['crs']

transform = proj['transform']
scale = None

studyArea = composites.first().geometry()

# Decompress composites by dividing by 10000 for optical bands and add indices
composites = composites.select(['blue','green','red','nir','swir1','swir2'])\
    .map(lambda img: img.divide(10000).float().copyProperties(img,['system:time_start']))
composites = composites.map(getImagesLib.simpleAddIndices)\
                      .map(getImagesLib.getTasseledCap)\
                      .map(getImagesLib.simpleAddTCAngles)


print('Done')

KeyError: 'startYear'

In [18]:
composites = composites.select(['blue','green','red','nir','swir1','swir2'])\
    .map(lambda img: img.divide(10000).float().copyProperties(img,['system:time_start']))
composites = composites.map(getImagesLib.simpleAddIndices)\
                      .map(getImagesLib.getTasseledCap)\
                      .map(getImagesLib.simpleAddTCAngles)

In [31]:
# Run map
vizParamsFalse = {\
  'min': 0.05,
  'max': [0.5,0.6,0.6],
  'bands': 'swir1,nir,red', #'bands': 'swir1,nir,red',   'bands': 'red,blue,green'
  'gamma': 1.6\
}
Map.port = 1996
Map.clearMap()
Map.addTimeLapse(clipped_collection,getImagesLib.vizParamsFalse,'Composites') #clipped_collection or composites
Map.centerObject(studyArea,10)
Map.setQueryDateFormat('YYYY')
Map.turnOnInspector()
Map.view()

Adding layer: Composites
Setting default query date format to: YYYY
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:1996/geeView/ already serving.
cwd /Users/jessieeastburn/Library/CloudStorage/OneDrive-UniversityofUtah/URSA_Work/TS2/PJ_TimeSeries
geeView URL: http://localhost:1996/geeView/?projectID=None&accessToken=ya29.a0AfB_byAwL6WpHsh6jy5YBSt0o82zv-XHu7ngI-qqn4SmZJ2x7lFYC3wUX8PjLajGFKvqAuqfnTv4JhkXDm7P0UqQa1jp9hA7jontowRORVLTRbIdh_NP-glUMG-Lr2qxz-tdB-XrFpDOrzqsiiXjpLW9XyG07LT1PKl-SAaCgYKAVQSARISFQHGX2MiMvA5Wg25tM6_NNGLC5smTA0173


::1 - - [21/Feb/2024 15:54:29] "GET /geeView/?projectID=None&accessToken=ya29.a0AfB_byAwL6WpHsh6jy5YBSt0o82zv-XHu7ngI-qqn4SmZJ2x7lFYC3wUX8PjLajGFKvqAuqfnTv4JhkXDm7P0UqQa1jp9hA7jontowRORVLTRbIdh_NP-glUMG-Lr2qxz-tdB-XrFpDOrzqsiiXjpLW9XyG07LT1PKl-SAaCgYKAVQSARISFQHGX2MiMvA5Wg25tM6_NNGLC5smTA0173 HTTP/1.1" 200 -
::1 - - [21/Feb/2024 15:54:29] "GET /geeView/src/gee/gee-run/runGeeViz.js HTTP/1.1" 200 -


In [23]:
#Example of how to get Landsat data using the getImagesLib, create median composites, run LandTrendr and then filter 
#LandTrendr output into usable data depicting where, when, and the magnitude of loss and gain
####################################################################################################
import os,sys
sys.path.append(os.getcwd())

#Module imports
try:
    import geeViz.getImagesLib as gil
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as gil


import geeViz.changeDetectionLib as cdl
ee = gil.ee

#Set up to mapper objects to use
#Can use the default one first
Map1 = gil.Map

#Set up another
Map2 = gil.mapper()

print('done')

done


In [25]:
# Define user parameters:

# Specify study area: Study area
# Can be a featureCollection, feature, or geometry
studyArea = studyArea

# Update the startJulian and endJulian variables to indicate your seasonal 
# constraints. This supports wrapping for tropics and southern hemisphere.
# If using wrapping and the majority of the days occur in the second year, the system:time_start will default 
# to June 1 of that year.Otherwise, all system:time_starts will default to June 1 of the given year
# startJulian: Starting Julian date 
# endJulian: Ending Julian date
startJulian = 1
endJulian = 365

# Specify start and end years for all analyses
# More than a 3 year span should be provided for time series methods to work 
# well. If using Fmask as the cloud/cloud shadow masking method, or providing
# pre-computed stats for cloudScore and TDOM, this does not 
# matter
startYear = 2015  
endYear = 2020


# Choose band or index
# NBR, NDMI, and NDVI tend to work best
# Other good options are wetness and tcAngleBG
indexName = 'NBR'

# How many significant loss and/or gain segments to include
# Do not make less than 1
# If you only want the first loss and/or gain, choose 1
# Generally any past 2 are noise
howManyToPull = 1

# Parameters to identify suitable LANDTRENDR segments

# Thresholds to identify loss in vegetation
# Any segment that has a change magnitude or slope less than both of these thresholds is omitted
lossMagThresh = -0.15
lossSlopeThresh = -0.05


# Thresholds to identify gain in vegetation
# Any segment that has a change magnitude or slope greater than both of these thresholds is omitted
gainMagThresh = 0.1
gainSlopeThresh = 0.05

slowLossDurationThresh = 3

# Choose from: 'newest','oldest','largest','smallest','steepest','mostGradual','shortest','longest'
chooseWhichLoss = 'largest'
chooseWhichGain = 'largest'

# LandTrendr Params
# run_params ={
#   'timeSeries': (ImageCollection) Yearly time-series from which to extract breakpoints. The first band is used to find breakpoints, and all subsequent bands are fitted using those breakpoints.
#   'maxSegments':            6,\ (Integer) Maximum number of segments to be fitted on the time series.
#   'spikeThreshold':         0.9,\ (Float, default: 0.9) Threshold for damping the spikes (1.0 means no damping).
#   'vertexCountOvershoot':   3,\(Integer, default: 3) The initial model can overshoot the maxSegments + 1 vertices by this amount. Later, it will be pruned down to maxSegments + 1.
#   'preventOneYearRecovery': False,\(Boolean, default: False): Prevent segments that represent one year recoveries.
#   'recoveryThreshold':      0.25,\(Float, default: 0.25) If a segment has a recovery rate faster than 1/recoveryThreshold (in years), then the segment is disallowed.
#   'pvalThreshold':          0.05,\(Float, default: 0.1) If the p-value of the fitted model exceeds this threshold, then the current model is discarded and another one is fitted using the Levenberg-Marquardt optimizer.
#   'bestModelProportion':    0.75,\(Float, default: 0.75) Allows models with more vertices to be chosen if their p-value is no more than (2 - bestModelProportion) times the p-value of the best model.
#   'minObservationsNeeded':  6\(Integer, default: 6) Min observations needed to perform output fitting.
# };

# Define landtrendr params
run_params = { \
  'maxSegments':            6,\
  'spikeThreshold':         0.9,\
  'vertexCountOvershoot':   3,\
  'preventOneYearRecovery': False,\
  'recoveryThreshold':      0.25,\
  'pvalThreshold':          0.05,\
  'bestModelProportion':    0.75,\
  'minObservationsNeeded':  6\
}

# Whether to add change outputs to map
addToMap = True

# Export params
# Whether to export LANDTRENDR change detection (loss and gain) outputs
exportLTLossGain = False

# Whether to export LandTrendr vertex array raw output
exportLTVertexArray = False

# Set up Names for the export
#outputName = 'LT_Test'

# Provide location composites will be exported to
# This should be an asset imageCollection
#exportPathRoot = 'users/username/someCollection'

# CRS- must be provided.  
# Common crs codes: Web mercator is EPSG:4326, USGS Albers is EPSG:5070, 
# WGS84 UTM N hemisphere is EPSG:326+ zone number (zone 12 N would be EPSG:32612) and S hemisphere is EPSG:327+ zone number
crs = 'EPSG:4326'

# Specify transform if scale is None and snapping to known grid is needed
transform = None

# Specify scale if transform is None
scale = None
####################################################################################################
#End user parameters
####################################################################################################
print('Done')

Done


In [ ]:
####################################################################################################
#Start function calls
####################################################################################################
#First, let's look at the Hansen Global Forest Change output
#This is a great product to get an idea of where loss has occurred 
Map1.port = 1231
#First clear the map in case it has been populated with layers/commands earlier
Map1.clearMap()

#Bring in Hansen data and add it to the map
hansen = ee.Image("UMD/hansen/global_forest_change_2022_v1_10").select(['lossyear']).add(2000).int16()
hansen = hansen.updateMask(hansen.neq(2000).And(hansen.gte(startYear)).And(hansen.lte(endYear)))
Map1.addLayer(hansen,{'min':startYear,'max':endYear,'palette':cdl.lossYearPalette},'Hansen Loss Year',True)

#Bring in map
Map1.turnOnInspector()
Map1.centerObject(studyArea)
Map1.view()
